In [ ]:
import pandas as pd
import numpy as np
import os
import re
import random
import pandas as pd

In [ ]:
import pandas as pd

# Specify the file path
file_path = r"/content/aws_review_sofware_dataset.csv"

# Initialize an empty DataFrame to store the first 5000 rows
df_5000_rows = pd.DataFrame()

# Read the dataset in chunks of, for example, 5000 rows
chunksize = 5000

# Iterate over chunks of the dataset
for chunk in pd.read_csv(file_path, chunksize=chunksize):
    # Concatenate chunks until we reach 5000 rows
    df_5000_rows = pd.concat([df_5000_rows, chunk])
    if len(df_5000_rows) >= 5000:
        # If we have reached or exceeded 5000 rows, break the loop
        df_5000_rows = df_5000_rows.iloc[:5000]  # Ensure only the first 1000 rows are kept
        break


In [ ]:
df_5000_rows.shape

(5000, 13)

In [ ]:
df_5000_rows.head()

,Unnamed: 0,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,0,4.0,True,"03 11, 2014",A240ORQ2LF9LUI,0077613252,{'Format:': ' Loose Leaf'},Michelle W,The materials arrived early and were in excell...,Material Great,1394496000,NaN,NaN
1,1,4.0,True,"02 23, 2014",A1YCCU0YRLS0FE,0077613252,{'Format:': ' Loose Leaf'},Rosalind White Ames,I am really enjoying this book with the worksh...,Health,1393113600,NaN,NaN
2,2,1.0,True,"02 17, 2014",A1BJHRQDYVAY2J,0077613252,{'Format:': ' Loose Leaf'},Allan R. Baker,"IF YOU ARE TAKING THIS CLASS DON""T WASTE YOUR ...",ARE YOU KIDING ME?,1392595200,7.0,NaN
3,3,3.0,True,"02 17, 2014",APRDVZ6QBIQXT,0077613252,{'Format:': ' Loose Leaf'},Lucy,This book was missing pages!!! Important pages...,missing pages!!,1392595200,3.0,NaN
4,4,5.0,False,"10 14, 2013",A2JZTTBSLS1QXV,0077775473,NaN,Albert V.,I have used LearnSmart and can officially say ...,Best study product out there!,1381708800,NaN,NaN


In [ ]:
df_5000_rows.columns

Index(['Unnamed: 0', 'overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'vote', 'image'],
      dtype='object')

In [ ]:
df_5000_rows["words"]="default value"
df_5000_rows["sentences"]="default value"


for i in range(df_5000_rows.shape[0]):
    df_5000_rows.at[i,"words"]= list("")
    df_5000_rows.at[i,"sentences"] = list("")


In [ ]:
from nltk.tokenize import sent_tokenize

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
for i in range(df_5000_rows.shape[0]):
    l1= sent_tokenize(str(df_5000_rows.loc[i,"reviewText"]))
    df_5000_rows.at[i,"sentences"]=l1

In [ ]:
!pip install pywsd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.6/31.6 MB 46.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 22.4 MB/s eta 0:00:00
  Created wheel for wn: filename=wn-0.0.23-py3-none-any.whl size=31792912 sha256=c3fb8f2110a9706c2268f28b90e0f4efafa165ed0d93a47ebd098f3f7c003677
  Stored in directory: /root/.cache/pip/wheels/a1/1a/7d/23a76ce45998af60e47466a694c237fa26023c5674b47672b2
Successfully built wn


In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from pywsd.utils import lemmatize_sentence

In [ ]:
for k in range(df_5000_rows.shape[0]):
    df_5000_rows.at[k,"words"]=list("")
    for j in range(len(df_5000_rows.loc[k,"sentences"])):
        df_5000_rows.at[k,"words"].extend(lemmatize_sentence(df_5000_rows.loc[k,"sentences"][j]))

In [ ]:
df_5000_rows["words_sentences"] = "default"

In [ ]:
import functools
for k in range(df_5000_rows.shape[0]):
    df_5000_rows.loc[k,"words_sentences"]=functools.reduce(lambda a,b:( str(a)+str(" ")+str(b)),df_5000_rows.loc[k,"words"])

In [ ]:
from sklearn.feature_extraction.text import  TfidfVectorizer

In [ ]:
df1=df_5000_rows

no_features = 1000
tf_vectorizer = TfidfVectorizer( max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(df1.words_sentences)

In [ ]:
#df_x=pd.DataFrame(tf.A, columns=tf_vectorizer.get_feature_names())
df_x = pd.DataFrame(tf.A, columns=tf_vectorizer.get_feature_names_out())

In [ ]:
df_y=df_5000_rows["verified"]

In [ ]:
df_y.head()

,verified
0,True
1,True
2,True
3,True
4,False


In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le=LabelEncoder()

In [ ]:
df_y_1=pd.DataFrame(df_y)

In [ ]:
df_y_encoded=df_y_1.apply(le.fit_transform)

In [ ]:
df_y_encoded.columns[0]

'verified'

In [ ]:
df_y_encoded.sample(5)

,verified
3204,0
1186,0
784,1
2562,1
1708,0


### GradientBoostingClassifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
GBC=GradientBoostingClassifier(n_estimators=3000)

In [ ]:
GBC.fit(df_x,df_y_enc)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(n_estimators=3000)

In [ ]:
GBC.score(df_x,df_y_enc)

0.9844

### NAIVE BAYES CLASSIFIER

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
NB=GaussianNB()

In [ ]:
NB.fit(df_x,df_y_enc)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB()

In [ ]:
NB.score(df_x,df_y_enc)

0.7402

### RANDOM FOREST CLASSIFIER

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RF=RandomForestClassifier(n_estimators=3000)

In [ ]:
RF.fit(df_x,df_y_encoded)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(n_estimators=3000)

In [ ]:
RF.score(df_x,df_y_encoded)

0.9996

### RANDOM FOREST CLASSIFIER

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_x, df_y_encoded, test_size=0.30, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RF1=RandomForestClassifier(n_estimators=3000)

In [ ]:
RF1.fit(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(n_estimators=3000)

In [ ]:
 y_pred=RF1.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_test,y_pred)

0.7753333333333333

In [ ]:
RF1.score(X_train,y_train)

0.9997142857142857

### GRADIENT BOOSTING CLASSIFIER

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier